In [1]:
#r "nuget:Microsoft.DotNet.Interactive.AI, 1.0.0-beta.23558.10"

Installed Packages Microsoft.DotNet.Interactive.AI, 1.0.0-beta.23468.5

Loading extensions from `C:\Users\dicolomb\.nuget\packages\skiasharp\2.88.5\interactive-extensions\dotnet\SkiaSharp.DotNet.Interactive.dll`

Loading extension script from `C:\Users\dicolomb\.nuget\packages\duckdb.interactiveextension\1.0.108\interactive-extensions\dotnet\extension.dib`

Query DuckDB databases. 
 This extension adds support for connecting to DuckDB databases using the #!connect duckdb magic command. ADO.NET support via DuckDB.NET project

Loading extension script from `C:\Users\dicolomb\.nuget\packages\microsoft.dotnet.interactive.ai\1.0.0-beta.23468.5\interactive-extensions\dotnet\extension.dib`

Added magic commands
 - `#!connect azure-openai`
 - `#!connect openai`
 - `#!connect huggingface`

In [2]:
#!value --name key
YOUR AZURE OPEN AI KEY

In [3]:
#!value --name endpoint
https://your-enpoint.openai.azure.com/

In [4]:
#!connect azure-openai --model-type TextEmbeddingGenerator --kernel-name knowledge --api-key @value:key --endpoint @value:endpoint --deployment text-embedding-ada-002

Kernel added: #!knowledge(duckDbVectorRAW)

Kernel added: #!knowledge(duckDbVector)

Kernel added: #!knowledge

In [5]:
#!connect azure-openai --model-type TextCompletion --kernel-name textCompletion --api-key @value:key --endpoint @value:endpoint --deployment text-davinci-003 --use-knowledge knowledge

Kernel added: #!textCompletion(skill)

Kernel added: #!textCompletion

In [6]:
#r "nuget: HtmlAgilityPack, 1.11.49"

Installed Packages HtmlAgilityPack, 1.11.49

The `SetTransformation` method is then called to set a transformation function that takes in text and a MIME type and returns a list of perspectives. The transformation function first checks if the MIME type is "text/html" and if so, extracts the plain text from the HTML using the `HtmlAgilityPack` library.

The function then uses the `Markdig` library to parse the input text into a Markdown document. The function filters the paragraphs from the Markdown document and generates a summary and overview for each paragraph using the `textKernel`. The take away points, summary, and overview are added to a list of perspectives.

If the length of the paragraph is greater than 1000 characters, the entire paragraph is added to the list of perspectives. The function returns the list of perspectives.

This code is useful for generating perspectives on input text using machine learning models. The `knowledgeKernel` will then use the data generated to populate the semantic memory and use text embeddings to retrieve relevant data to focus questions sent to the `TextCompletionKernel`.

```mermaid 
sequenceDiagram
    participant User
    participant KnowledgeKernel
    participant TextCompletionKernel

    User->>KnowledgeKernel: Sends text to be stored in the KnowledgeKernel
    KnowledgeKernel->>KnowledgeKernel: Extracts plain text from input
    KnowledgeKernel->>KnowledgeKernel: Parses input using MarkdownPipelineBuilder
    KnowledgeKernel->>KnowledgeKernel: Filters paragraphs from the parsed input
    loop Processing each paragraph 
        KnowledgeKernel->>TextCompletionKernel: Sends a prompt to summarize the paragraph
        TextCompletionKernel->>KnowledgeKernel: Extracts the take away points from the summary
        KnowledgeKernel->>TextCompletionKernel: Sends a prompt to summarize the paragraph in 5 lines
        TextCompletionKernel->>KnowledgeKernel: Extracts the summary from the prompt
        KnowledgeKernel->>TextCompletionKernel: Sends a prompt to provide the most important information
        TextCompletionKernel->>KnowledgeKernel: Extracts the overview from the prompt
    end
    KnowledgeKernel->>User: Returns the take away points, summary, and overview
```

In [7]:
using Microsoft.DotNet.Interactive.AI;
using Microsoft.DotNet.Interactive;
using Microsoft.DotNet.Interactive.Formatting;
using Microsoft.DotNet.Interactive.Commands;
using Microsoft.DotNet.Interactive.Events;
using HtmlAgilityPack;
using Markdig;
using Markdig.Helpers;
using Markdig.Parsers;
using Markdig.Parsers.Inlines;
using Markdig.Syntax;

List<(string text, Dictionary<string,object>? metadata)> lastData = null;

var knowledgeKernel = Kernel.Root.FindKernelByName("knowledge") as KnowledgeKernel;

var textKernel = Kernel.Root.FindKernelByName("textCompletion") as TextCompletionKernel;

knowledgeKernel.SetTransformation(async (text,mimeType)=>{
    textKernel.SetSuppressDisplay(true);
    var input = text;
    if(mimeType == "text/html"){
        var doc = new HtmlDocument();
        doc.LoadHtml(text);
        input = doc.DocumentNode.InnerText;
    }

    var pipeline = new MarkdownPipelineBuilder().UseAdvancedExtensions().Build();


    var document = Markdown.Parse(input, pipeline);
    var  paragraphs = document.Where(d => d is ParagraphBlock);
    var perspectives =  new List<(string text, Dictionary<string,object>? metadata)>();
    foreach (var paragraph in paragraphs)
    {
        var block = input.Substring( paragraph.Span.Start, paragraph.Span.Length);
        var result = await textKernel.SendAsync(new SubmitCode(
$"""
summararize the following text providing the take away points:
{block}
"""));

        var takeAway = result.Events.OfType<ReturnValueProduced>().LastOrDefault()?.FormattedValues.First(f => f.MimeType == PlainTextFormatter.MimeType).Value.Trim();  
    
        result = await textKernel.SendAsync(new SubmitCode(
$"""
 summararize the following text with up to 5 lines
{block}
"""));
        
        var summary = result.Events.OfType<ReturnValueProduced>().LastOrDefault()?.FormattedValues.First(f => f.MimeType == PlainTextFormatter.MimeType).Value.Trim();  

        result = await textKernel.SendAsync(new SubmitCode(
$"""
given
{takeAway}

and 

{block}

what is the most important information to know?
"""));
        
        var overView = result.Events.OfType<ReturnValueProduced>().LastOrDefault()?.FormattedValues.First(f => f.MimeType == PlainTextFormatter.MimeType).Value.Trim();  


        perspectives.Add((takeAway, null));
        perspectives.Add((summary, null));
        perspectives.Add((overView, null));

        // comment the following lines to avoid printing all results 
        $"""
        Storing derived knowledge: 
        takeAway 
        ---
        {takeAway}
        ---

        summary---
        {summary}
        ---

        overView---
        {overView}
        ---

        """.Display();

        if (block.Length > 1000)
        {
            perspectives.Add((block, null));
        }
    }   
    lastData = perspectives;
    textKernel.SetSuppressDisplay(false);
    return perspectives;
});

## Sore facts into the knowledge

You can skip this step if you just want to load persisted vector store

In [8]:
#!knowledge --name fact1
.NET Interactive is a polyglot kernel.

Storing derived knowledge: 
takeAway 
---
It allows you to write and execute code in multiple languages such as C#, F#, PowerShell, Python, and SQL.

Take Away Points:
1. .NET Interactive is a polyglot kernel.
2. It allows users to write and execute code in multiple languages, including C#, F#, PowerShell, Python, and SQL.
---

summary---
It allows users to write code in multiple languages, including C#, F#, PowerShell, and Python. It also provides a rich set of APIs for working with .NET, including the .NET Compiler Platform (“Roslyn”) and the .NET Runtime. It is designed to be used in the browser, in the terminal, or in a notebook.

.NET Interactive is a powerful tool for developers, allowing them to write code in multiple languages and access a rich set of APIs. It supports C#, F#, PowerShell, and Python, as well as the .NET Compiler Platform and .NET Runtime. It can be used in the browser, terminal, or notebook.
---

overView---
The most important information to know about .NET Int

In [14]:

#!knowledge --name fact2
.NET Interactive is developed by a team of engineers at Microsoft.

#!knowledge --name fact3
.NET Interactive is an open source project with a lot of contributors.

#!knowledge --name fact4
.NET Interactive is used by engineers.

#!knowledge --name fact5
.NET Interactive is developed by open source community.

#!knowledge --name fact6
.NET Interactive is used by software engineers.

#!knowledge --name fact7
.NET Interactive can create rich outputs and visualizations.

#!knowledge --name fact8
The Polyglot Notebooks extension, powered by .NET Interactive, brings support for multi-language notebooks to Visual Studio Code. 
Classic notebook software typically supports notebooks that use only one language at a time. With Polyglot Notebooks, features such as completions, documentation, syntax highlighting, and diagnostics are available for many languages in one notebook. In addition, different cells in the same notebook can run in separate processes or on different machines, allowing a notebook to span local and cloud environments in one combined workflow.
Polyglot Notebooks are fully interoperable with Jupyter and support the .ipynb file extension. 
You don't need to choose between the capabilities of Polyglot Notebooks and the rich Jupyter ecosystem. 
If your notebook is saved in the .ipynb format, you can open it in Jupyter and the cell languages will still be recognized. 
When working in Jupyter using the .NET Interactive kernel, you can switch cell languages using magic commands.

#!knowledge --name fact9
.NET Interactive has support for mermaid langauge and javascript. Along with the HTML kernel these components can be used to create rich visualizations. With the mermaid language you can create diagrams and flowcharts. With javascript you can create interactive visualizations. With HTML you can create rich visualizations.

#!knowledge --name fact10
.NET Interactive alloes using languages like Python, R and Julia thanks to the Jupyter protocol support. This allows you to use the rich ecosystem of these languages in your .NET Interactive notebooks.

Storing derived knowledge: 
takeAway 
---
It is a polyglot kernel that enables users to write and execute code in multiple languages, such as C#, F#, PowerShell, Python, and SQL. It is an open source project, allowing users to contribute to its development. It is a powerful and versatile platform for developers to create and test code, and a great tool for developers to quickly test and debug code.

Take Away Points:
1. .NET Interactive is a polyglot kernel developed by a team of engineers at Microsoft.
2. It allows users to write and execute code in multiple languages, including C#, F#, PowerShell, Python, and SQL.
3. It is an open source project, allowing users to contribute to its development.
4. It is a powerful and versatile platform for developers to create and test code.
5. It is a great tool for developers to quickly test and debug code.
---

summary---
It is an open source project that enables users to write and execute code in multiple languages, such as C#, F#, PowerShell, P

Storing derived knowledge: 
takeAway 
---
Take Away Points:
1. .NET Interactive is an open source project with a lot of contributors.
2. It allows users to write and execute code in multiple languages, including C#, F#, PowerShell, Python, and SQL.
3. It provides a powerful and versatile platform for developers to create and test code.
4. It is a great tool for developers to quickly test and debug code.
5. It is an invaluable resource for developers to learn and explore new technologies.
---

summary---
It enables users to write and execute code in multiple languages, such as C#, F#, PowerShell, Python, and SQL.
It provides a powerful and versatile platform for developers to create and test code.
It is a great tool for developers to quickly test and debug code.
It is an invaluable resource for developers to learn and explore new technologies.
---

overView---
The most important information to know about .NET Interactive is that it allows users to write and execute code in multiple lang

Storing derived knowledge: 
takeAway 
---
Take Away Points:
1. .NET Interactive is an open source project with a lot of contributors.
2. It allows users to write and execute code in multiple languages, including C#, F#, PowerShell, Python, and SQL.
3. It is used by engineers.
---

summary---
It is an open source project with many contributors.
It enables users to write and execute code in multiple languages.
It provides a powerful platform for developers to create and debug code.
It is a great resource for learning and exploring new technologies.
---

overView---
The most important information to know about .NET Interactive is that it allows users to write and execute code in multiple languages, including C#, F#, PowerShell, Python, and SQL.
---


Storing derived knowledge: 
takeAway 
---
Take Away Points:
1. .NET Interactive is an open source project with a lot of contributors.
2. It allows users to write and execute code in multiple languages, including C#, F#, PowerShell, Python, and SQL.
3. It is developed by an open source community.
---

summary---
It enables users to write and execute code in multiple languages, such as C#, F#, PowerShell, Python, and SQL.
It is a powerful and versatile platform for developers to create and test code.
It is a great tool for developers to quickly test and debug code.
It is an invaluable resource for developers to learn and explore new technologies.
---

overView---
The most important information to know about .NET Interactive is that it allows users to write and execute code in multiple languages, including C#, F#, PowerShell, Python, and SQL.
---


Storing derived knowledge: 
takeAway 
---
Take Away Points:
1. .NET Interactive is an open source project with a lot of contributors.
2. It allows users to write and execute code in multiple languages, including C#, F#, PowerShell, Python, and SQL.
3. It is used by software engineers.
---

summary---
It is an open source project with many contributors.
It enables users to write and execute code in multiple languages, such as C#, F#, PowerShell, Python, and SQL.
It provides a powerful and versatile platform for developers to create and test code.
It is a great tool for quickly testing and debugging code, as well as learning and exploring new technologies.
---

overView---
The most important information to know is that .NET Interactive is an open source project with a lot of contributors, allowing users to write and execute code in multiple languages, including C#, F#, PowerShell, Python, and SQL, and is used by software engineers.
---


Storing derived knowledge: 
takeAway 
---
Take Away Points:
1. .NET Interactive is a polyglot kernel.
2. It allows users to write and execute code in multiple languages, including C#, F#, PowerShell, Python, and SQL.
3. It can create rich outputs and visualizations.
---

summary---
It is an open source project with many contributors, and it enables developers to write and execute code in multiple languages. It is a great tool for quickly testing and debugging code, and it provides an invaluable resource for learning and exploring new technologies.
---

overView---
The most important information to know about .NET Interactive is that it is a polyglot kernel that allows users to write and execute code in multiple languages, including C#, F#, PowerShell, Python, and SQL, and that it can create rich outputs and visualizations.
---


Storing derived knowledge: 
takeAway 
---
Take Away Points:
1. The Polyglot Notebooks extension, powered by .NET Interactive, brings support for multi-language notebooks to Visual Studio Code.
2. It allows features such as completions, documentation, syntax highlighting, and diagnostics to be available for many languages in one notebook.
3. Different cells in the same notebook can run in separate processes or on different machines, allowing a notebook to span local and cloud environments in one combined workflow.
4. Polyglot Notebooks are fully interoperable with Jupyter and support the .ipynb file extension.
5. If your notebook is saved in the .ipynb format, you can open it in Jupyter and the cell languages will still be recognized.
6. When working in Jupyter using the .NET Interactive kernel, you can switch cell languages using magic commands.
---

summary---
Take Away Points:
1. Polyglot Notebooks is an extension powered by .NET Interactive that supports multi-language notebooks in 

Storing derived knowledge: 
takeAway 
---
Take Away Points:
1. .NET Interactive supports mermaid language, javascript, and HTML kernels.
2. These components can be used to create rich visualizations.
3. Mermaid language can be used to create diagrams and flowcharts.
4. Javascript can be used to create interactive visualizations.
5. HTML can be used to create rich visualizations.
---

summary---
.NET Interactive is a polyglot kernel that enables users to write and execute code in multiple languages, including C#, F#, PowerShell, Python, and SQL. It can create rich outputs and visualizations, as well as diagrams and flowcharts with the mermaid language, interactive visualizations with javascript, and HTML-based visualizations.
---

overView---
The most important information to know about .NET Interactive is that it is a polyglot kernel that allows users to write and execute code in multiple languages, including C#, F#, PowerShell, Python, and SQL, and that it has support for mermaid lang

Storing derived knowledge: 
takeAway 
---
Take Away Points:
1. .NET Interactive supports the Jupyter protocol, allowing users to write and execute code in languages such as Python, R, and Julia.
2. This provides access to the rich ecosystems of these languages in .NET Interactive notebooks.
---

summary---
It also supports mermaid language, javascript, and HTML kernels which can be used to create rich visualizations. With .NET Interactive, you can write and execute code in multiple languages, including C#, F#, PowerShell, Python, and SQL.
---

overView---
The most important information to know is that .NET Interactive supports the Jupyter protocol, allowing users to write and execute code in languages such as Python, R, and Julia, providing access to the rich ecosystems of these languages in .NET Interactive notebooks.
---


In [15]:
#!knowledge --name fact20 --from-url https://raw.githubusercontent.com/dotnet/interactive/main/docs/FAQ.md

Storing derived knowledge: 
takeAway 
---
-1 Take Away Points:
1. Notebooks are a type of program that allow mixing formatted text and executable code to create documents with runnable examples.
2. Notebooks are an example of literate programming.
3. Notebooks have cells which are different text regions, typically including code, markdown, and output.
---

summary---
1. .NET Interactive and the Polyglot Notebooks extension allow users to write and execute code in multiple languages in one notebook, and are interoperable with Jupyter and support the .ipynb file extension. 
2. Cells in the same notebook can run in separate processes or on different machines, allowing a notebook to span local and cloud environments in one combined workflow.
3. .NET Interactive supports the Jupyter protocol, providing access to the rich ecosystems of languages such as Python, R, and Julia in .NET Interactive notebooks.
---

overView---
The most important information to know about computational notebooks is

Storing derived knowledge: 
takeAway 
---
Take Away Points:
1. Project Jupyter is a popular notebook technology used for data analysis and data science.
2. It is also used for interactive documentation, learning materials, troubleshooting guides, and self-guiding automation scripts.
3. It can capture structured or visual log output.
---

summary---
.NET Interactive and the Polyglot Notebooks extension bring support for multi-language notebooks to Visual Studio Code, allowing features such as completions, documentation, syntax highlighting, and diagnostics to be available for many languages in one notebook. It also allows different cells in the same notebook to run in separate processes or on different machines, allowing a notebook to span local and cloud environments in one combined workflow. It is fully interoperable with Jupyter and supports the .ipynb file extension.

Project Jupyter is a popular notebook technology used for data analysis, data science, interactive documentation, tr

Storing derived knowledge: 
takeAway 
---
Take Away Points:
1. Jupyter Notebook is a browser-based notebook UI from Project Jupyter.
2. It can edit and run notebooks using a Jupyter kernel and the .ipynb file format.
---

summary---
.NET Interactive and the Polyglot Notebooks extension allow users to write and execute code in multiple languages in one notebook, and are interoperable with Jupyter. Cells in the same notebook can run in separate processes or on different machines, and features such as completions, documentation, syntax highlighting, and diagnostics are available for many languages. Magic commands can be used to switch cell languages in Jupyter.
---

overView---
The most important information to know about Jupyter Notebook is that it is a browser-based notebook UI from Project Jupyter that can edit and run notebooks using a Jupyter kernel and the .ipynb file format.
---


Storing derived knowledge: 
takeAway 
---
-Take Away Points:
1. JupyterLab is a browser-based frontend for Jupyter kernels.
2. It is similar to Jupyter Notebook, but richer and more extensible.
---

summary---
.NET Interactive and the Polyglot Notebooks extension allow users to write and execute code in multiple languages in one notebook, and are interoperable with Jupyter and support the .ipynb file extension. It also enables cells in the same notebook to run in separate processes or on different machines, allowing a notebook to span local and cloud environments. It supports mermaid language, javascript, and HTML kernels for creating rich visualizations. JupyterLab is a browser-based frontend for Jupyter kernels, similar to Jupyter Notebook, but more extensible.
---

overView---
The most important information to know about JupyterLab is that it is a browser-based frontend for Jupyter kernels, similar to Jupyter Notebook, but richer and more extensible.
---


Storing derived knowledge: 
takeAway 
---
-Take Away Points:
1. The term "frontend" is used to describe the UI for a notebook editor.
2. Examples of notebook frontends include Jupyter Notebook, JupyterLab, nteract Desktop, and the Jupyter Extension for VS Code and GitHub Codespaces.
---

summary---
.NET Interactive and the Polyglot Notebooks extension allow users to write and execute code in multiple languages in one notebook, and are interoperable with Jupyter and support the .ipynb file extension. It also allows different cells in the same notebook to run in separate processes or on different machines, enabling a notebook to span local and cloud environments. Additionally, .NET Interactive supports mermaid language, javascript, and HTML kernels for creating rich visualizations.
---

overView---
The most important information to know is that the term "frontend" is used to describe the UI for a notebook editor, and that there are several popular notebook frontends available, such as Ju

...

Storing derived knowledge: 
takeAway 
---
-Take Away Points:
1. VS Code needs to be updated when a "Restart to Update" message appears in the menu.
---

summary---
1. Visual Studio Code needs to be updated when a "Restart to Update" message appears in the menu.
---

overView---
The most important information to know is that when you see a "Restart to Update" message in the menu, VS Code needs to be updated.
---


Storing derived knowledge: 
takeAway 
---
-Take Away Points:
1. The Polyglot Notebooks extension may require a reload after a VS Code update.
---

summary---
The Polyglot Notebooks extension, powered by .NET Interactive, brings support for multi-language notebooks to Visual Studio Code. It allows features such as completions, documentation, syntax highlighting, and diagnostics to be available for many languages in one notebook. It is interoperable with Jupyter and supports the .ipynb file extension, allowing cells in the same notebook to run in separate processes or on different machines. It may require a reload after a VS Code update.
---

overView---
The most important information to know about the Polyglot Notebooks extension is that it may require a reload after a VS Code update.
---


Storing derived knowledge: 
takeAway 
---
Take Away Points:
1. If you are having issues running code, open an issue on the .NET Interactive GitHub page.
2. The .NET Interactive GitHub page provides a platform to report issues and get help.
---

summary---
.NET Interactive is an open source project with a lot of contributors, allowing users to write and execute code in multiple languages, including C#, F#, PowerShell, Python, and SQL. It is a great tool for developers to quickly test and debug code, and an invaluable resource for developers to learn and explore new technologies. If you encounter any issues, please open an issue on GitHub.
---

overView---
The most important information to know is that if you are having issues running code, you should open an issue on the .NET Interactive GitHub page to get help.
---


Storing derived knowledge: 
takeAway 
---
-Take Away Points:
1. An error can occur when calling a .NET Interactive formatting API such as `Display`, `#!share`, or `#!set`.
2. This error occurs when formatting the specified object using the requested MIME type isn't available.
3. Formatters for additional MIME types can be added using the formatter registration APIs.
4. More details about these APIs can be found in the formatting.md document.
---

summary---
.NET Interactive is a polyglot kernel that allows users to write and execute code in multiple languages, including C#, F#, PowerShell, Python, and SQL. It also supports mermaid language, javascript, and HTML kernels which can be used to create rich visualizations. It supports the Jupyter protocol, providing access to the rich ecosystems of these languages in .NET Interactive notebooks. Formatting APIs such as `Display`, `#!share` and `#!set` can be used to add formatters for additional MIME types.
---

overView---
The most important

Storing derived knowledge: 
takeAway 
---
Take Away Points:
1. An error is shown when a formatted value from the .NET Interactive kernel is formatted using a MIME type that VS Code doesn't know how to render.
2. VS Code can only render notebook outputs in MIME types for which there's an installed notebook renderer.
3. Notebook renderers are independent of the kernel and can be installed from the Visual Studio Marketplace.
---

summary---
.NET Interactive and the Polyglot Notebooks extension allow users to write and execute code in multiple languages in one notebook, and are interoperable with Jupyter and support the .ipynb file extension. Cells in the same notebook can run in separate processes or on different machines, allowing a notebook to span local and cloud environments in one combined workflow. Features such as completions, documentation, syntax highlighting, and diagnostics are available for many languages in one notebook.
---

overView---
The most important information to know

In [16]:
SHOW TABLES;

name
defaultVectorCollection


In [17]:
select id, payload from defaultVectorCollection limit 5;

id,payload
fact1-0,"Take Away Points: 1. .NET Interactive is a polyglot kernel. 2. It allows users to write and execute code in multiple languages, including C#, F#, PowerShell, Python, and SQL."
fact1-1,"It enables users to write and execute code in multiple languages, such as C#, F#, PowerShell, Python, and SQL. It provides a powerful and versatile platform for developers to create and test code. It is an open source project, allowing users to contribute to its development. It is a great tool for developers to quickly test and debug code."
fact1-2,"The most important information to know about .NET Interactive is that it allows users to write and execute code in multiple languages, including C#, F#, PowerShell, Python, and SQL."
fact2-0,"It is a polyglot kernel that enables users to write and execute code in multiple languages, such as C#, F#, PowerShell, Python, and SQL. It is an open source project, allowing users to contribute to its development. It is a powerful and versatile platform for developers to create and test code, and a great tool for developers to quickly test and debug code. Take Away Points: 1. .NET Interactive is a polyglot kernel developed by a team of engineers at Microsoft. 2. It allows users to write and execute code in multiple languages, including C#, F#, PowerShell, Python, and SQL. 3. It is an open source project, allowing users to contribute to its development. 4. It is a powerful and versatile platform for developers to create and test code. 5. It is a great tool for developers to quickly test and debug code."
fact2-1,"It is an open source project that enables users to write and execute code in multiple languages, such as C#, F#, PowerShell, Python, and SQL. It provides a powerful and versatile platform for developers to create and test code. It is a great tool for developers to quickly test and debug code. It is an invaluable resource for developers to learn and explore new technologies."


In [10]:
using Microsoft.DotNet.Interactive.AI.DuckDb;
using System.IO;
var currentDirectory = Directory.GetCurrentDirectory();
var dataFile = Path.Combine(currentDirectory, "interactive_knowledge.json");


## can export and import the vectors stored by the `knowledge kernel`

In [19]:
using Microsoft.DotNet.Interactive.AI.DuckDb;
await Kernel.Root.SendAsync(new ExportCollection("defaultVectorCollection", dataFile, "knowledge(duckDbVector)"));

## use the following cell to just load the persisted knowledge

In [11]:
using Microsoft.DotNet.Interactive.AI.DuckDb;
await Kernel.Root.SendAsync(new ImportCollection("defaultVectorCollection", dataFile, "knowledge(duckDbVector)"));

## use the knowlegde to augment prompts (RAG)

In [12]:
what can i do with .NET Interactive?

With .NET Interactive, you can write and execute code in multiple languages, including C# Script, F#, PowerShell, Python, and SQL. You can also create rich visualizations such as diagrams, flowcharts, interactive visualizations, and HTML visualizations using mermaid language, javascript, and HTML kernels. Additionally, you can extend the capabilities of .NET Interactive with custom magic commands using these languages.

In [13]:

does it  support  python?

Yes, .NET Interactive supports the Jupyter protocol, allowing users to write and execute code in languages such as Python, R, and Julia.

In [14]:
#!function be_clear
Answer the following questions:
{{$input}}

And make sure to be succint and clear.

In [15]:
#!use-skills function._GLOBAL_FUNCTIONS_.be_clear
Why can I do data visualizations with .NET Interactive?

Yes, .NET Interactive allows users to create rich visualizations using mermaid language, javascript, and HTML kernels. It also provides extension APIs to create custom magic commands and execute code in multiple languages.

In [16]:
What should I do if a cell runs forever?

If a cell runs forever, you should open an issue on the .NET Interactive GitHub page to get help. The .NET Interactive GitHub page provides a platform to report issues and get help. You can also use the Magic commands to switch cell languages in Jupyter.

In [17]:
Can I create custom kernels for langauges and tools that are not supported by the Jupyter Notebook?

Yes, you can create custom kernels for languages and tools that are not supported by the Jupyter Notebook. .NET Interactive and the Polyglot Notebooks extension allow users to write and execute code in multiple languages in one notebook, and are interoperable with Jupyter and support the .ipynb file extension. Cells in the same notebook can run in separate processes or on different machines, allowing a notebook to span local and cloud environments in one combined workflow. It also supports mermaid language, javascript, and HTML kernels for creating rich visualizations. JupyterLab is a browser-based frontend for Jupyter kernels, similar to Jupyter Notebook, but more extensible.